In [1]:
import sqlite3
import urllib.request
import pandas as pd

# Download database (if needed)
url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
urllib.request.urlretrieve(url, "Chinook.sqlite")

# Connect to the database
conn = sqlite3.connect("Chinook.sqlite")
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in Chinook database:")
pd.DataFrame(tables, columns=['Employee'])



Tables in Chinook database:


,Employee
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [3]:
print("\n--- Verifying Queries ---")

# Query 1: Fetch first 5 employees
query1 = "SELECT * FROM Employee LIMIT 5;"
df1 = pd.read_sql_query(query1, conn)
print("\nFirst 5 Employees:")
print(df1)

# Query 2: Fetch first 5 invoices
query2 = "SELECT * FROM Invoice LIMIT 5;"
df2 = pd.read_sql_query(query2, conn)
print("\nFirst 5 Invoices:")
print(df2)

# Query 3: Fetch first 5 invoices from USA
query3 = "SELECT * FROM Invoice WHERE BillingCountry = 'USA' LIMIT 5;"
df3 = pd.read_sql_query(query3, conn)
print("\nFirst 5 Invoices from USA:")
print(df3)


--- Verifying Queries ---

First 5 Employees:
   EmployeeId LastName FirstName                Title  ReportsTo  \
0           1    Adams    Andrew      General Manager        NaN   
1           2  Edwards     Nancy        Sales Manager        1.0   
2           3  Peacock      Jane  Sales Support Agent        2.0   
3           4     Park  Margaret  Sales Support Agent        2.0   
4           5  Johnson     Steve  Sales Support Agent        2.0   

             BirthDate             HireDate              Address      City  \
0  1962-02-18 00:00:00  2002-08-14 00:00:00  11120 Jasper Ave NW  Edmonton   
1  1958-12-08 00:00:00  2002-05-01 00:00:00         825 8 Ave SW   Calgary   
2  1973-08-29 00:00:00  2002-04-01 00:00:00        1111 6 Ave SW   Calgary   
3  1947-09-19 00:00:00  2003-05-03 00:00:00     683 10 Street SW   Calgary   
4  1965-03-03 00:00:00  2003-10-17 00:00:00         7727B 41 Ave   Calgary   

  State Country PostalCode              Phone                Fax  \
0    AB

In [4]:
# 3. Query across Employee, Customer, and Invoice tables (filtering invoices from USA)
print("\n--- Join Across Employee, Customer, and Invoice Tables ---")

query4 = """
SELECT e.FirstName AS EmployeeFirstName, e.LastName AS EmployeeLastName, 
       c.FirstName AS CustomerFirstName, c.LastName AS CustomerLastName, 
       i.InvoiceDate, i.Total
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId  -- Employee manages Customer
JOIN Invoice i ON c.CustomerId = i.CustomerId  -- Customer makes Invoices
WHERE c.Country = 'USA'
LIMIT 10;
"""

df4 = pd.read_sql_query(query4, conn)
print("\nEmployee, Customer, Invoice Date, and Total from USA:")
print(df4)



--- Join Across Employee, Customer, and Invoice Tables ---

Employee, Customer, Invoice Date, and Total from USA:
  EmployeeFirstName EmployeeLastName CustomerFirstName CustomerLastName  \
0          Margaret             Park             Frank           Harris   
1          Margaret             Park             Frank           Harris   
2          Margaret             Park             Frank           Harris   
3          Margaret             Park             Frank           Harris   
4          Margaret             Park             Frank           Harris   
5          Margaret             Park             Frank           Harris   
6          Margaret             Park             Frank           Harris   
7             Steve          Johnson              Jack            Smith   
8             Steve          Johnson              Jack            Smith   
9             Steve          Johnson              Jack            Smith   

           InvoiceDate  Total  
0  2009-02-19 00:00:00   0.

In [14]:


# Drop indexes that optimize the joins
cursor.executescript("""
DROP INDEX IF EXISTS IFK_CustomerInvoice;
DROP INDEX IF EXISTS IFK_InvoiceLineInvoiceId;
DROP INDEX IF EXISTS IFK_InvoiceLineTrackId;
DROP INDEX IF EXISTS IFK_CustomerSupportRep;  -- Drop the index on SupportRepId
""")
conn.commit()

# Define our unindexed join query (with Employee instead of Customer)
unindexed_query = """
SELECT e.FirstName, e.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId  -- Employee linked to Customer via SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId    -- Customer linked to Invoice
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId  -- Invoice linked to InvoiceLine
JOIN Track t ON ii.TrackId = t.TrackId            -- InvoiceLine linked to Track
LIMIT 10;
"""

# Time the unindexed query
times_unindexed = []
for _ in range(10):
    # Vary the limit to avoid cache effects
    current_limit = random.randint(5, 15)
    varied_query = unindexed_query.replace('LIMIT 10', f'LIMIT {current_limit}')
    
    start_time = time.time()
    df_unindexed = pd.read_sql_query(varied_query, conn)
    end_time = time.time()
    times_unindexed.append(end_time - start_time)

# Recreate indexes to optimize performance
cursor.executescript("""
CREATE INDEX IF NOT EXISTS IFK_CustomerSupportRep ON Customer(SupportRepId);
CREATE INDEX IF NOT EXISTS IFK_CustomerInvoice ON Invoice(CustomerId);
CREATE INDEX IF NOT EXISTS IFK_InvoiceLineInvoiceId ON InvoiceLine(InvoiceId);
CREATE INDEX IF NOT EXISTS IFK_InvoiceLineTrackId ON InvoiceLine(TrackId);
""")
conn.commit()

# Print results
print(f"Unindexed join average time: {sum(times_unindexed)/len(times_unindexed):.4f} seconds")
print(f"Individual times: {[round(t, 4) for t in times_unindexed]}")

# Close the connection
conn.close()


DatabaseError: Execution failed on sql '
SELECT e.FirstName, e.LastName, i.InvoiceDate, t.Name AS TrackName
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId  -- Employee linked to Customer via SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId    -- Customer linked to Invoice
JOIN InvoiceLine ii ON i.InvoiceId = ii.InvoiceId  -- Invoice linked to InvoiceLine
JOIN Track t ON ii.TrackId = t.TrackId            -- InvoiceLine linked to Track
LIMIT 10;
': no such table: Employee

In [5]:
# 5. Time the query from #4 (Joining Employee, Customer, and Invoice)
print("\n--- Timing Unindexed Query (10 Runs) ---")

import time
import random

times = []

query5 = """
SELECT e.FirstName AS EmployeeFirstName, e.LastName AS EmployeeLastName, 
       c.FirstName AS CustomerFirstName, c.LastName AS CustomerLastName, 
       i.InvoiceDate, i.Total
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId  -- Employee manages Customer
JOIN Invoice i ON c.CustomerId = i.CustomerId  -- Customer makes Invoices
WHERE c.Country = 'USA'
LIMIT 10;
"""

for _ in range(10):
    start_time = time.time()
    df5 = pd.read_sql_query(query5, conn)
    end_time = time.time()
    times.append(end_time - start_time)

    # Vary the limit to avoid cache effects
    query5_vary = query5.replace('LIMIT 10', f'LIMIT {random.randint(5, 15)}')

print(f"Average time: {sum(times) / len(times)} seconds")



--- Timing Unindexed Query (10 Runs) ---
Average time: 0.0008584976196289063 seconds


In [6]:
# 6. Create indexes for Employee, Customer, and Invoice tables to improve query performance
print("\n--- Creating Indexes ---")

try:
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_EmployeeId ON Employee (EmployeeId);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_SupportRepId ON Customer (SupportRepId);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_CustomerId ON Customer (CustomerId);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_InvoiceId ON Invoice (InvoiceId);")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_Invoice_CustomerId ON Invoice (CustomerId);")
    
    conn.commit()
    print("Indexes created successfully.")
except Exception as e:
    print("Error creating indexes:", e)



--- Creating Indexes ---
Indexes created successfully.


In [7]:
# 7. Time the query again after adding indexes
print("\n--- Timing Indexed Query (10 Runs) ---")

query6 = """
SELECT e.FirstName AS EmployeeFirstName, e.LastName AS EmployeeLastName, 
       c.FirstName AS CustomerFirstName, c.LastName AS CustomerLastName, 
       i.InvoiceDate, i.Total
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId  -- Employee manages Customer
JOIN Invoice i ON c.CustomerId = i.CustomerId  -- Customer makes Invoices
WHERE c.Country = 'USA'
LIMIT 10;
"""

times_indexed = []
for _ in range(10):
    start_time = time.time()
    df6 = pd.read_sql_query(query6, conn)
    end_time = time.time()
    times_indexed.append(end_time - start_time)
    
    # Vary the limit slightly to reduce cache effects
    query6_vary = query6.replace('LIMIT 10', f'LIMIT {random.randint(5, 15)}')

print(f"Average time (indexed): {sum(times_indexed) / len(times_indexed)} seconds")



--- Timing Indexed Query (10 Runs) ---
Average time (indexed): 0.000841522216796875 seconds


In [8]:
# 8. Look at the query plan to make sure the index capabilities are being used.
print("\n--- Query Plan ---")

query6 = """
SELECT e.FirstName, e.LastName, c.FirstName AS CustomerFirstName, c.LastName AS CustomerLastName, 
       i.InvoiceDate, i.Total
FROM Employee e
JOIN Customer c ON e.EmployeeId = c.SupportRepId
JOIN Invoice i ON c.CustomerId = i.CustomerId
LIMIT 10;
"""

cursor.execute("EXPLAIN QUERY PLAN " + query6)
query_plan = cursor.fetchall()
print(pd.DataFrame(query_plan))



--- Query Plan ---
    0  1  2                                                  3
0   5  0  0                            SCAN TABLE Invoice AS i
1   7  0  0  SEARCH TABLE Customer AS c USING INTEGER PRIMA...
2  10  0  0  SEARCH TABLE Employee AS e USING INTEGER PRIMA...


In [16]:
conn.close()